In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
import gensim.downloader as api

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier


from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import GridSearchCV


In [4]:
Wv = api.load('word2vec-google-news-300')

In [5]:
df=pd.read_csv("all_kindle_review.csv")

In [6]:
df

,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000
...,...,...,...,...,...,...,...,...,...,...,...
11995,11995,2183,B001DUGORO,"[0, 0]",4,Valentine cupid is a vampire- Jena and Ian ano...,"02 28, 2014",A1OKS5Q1HD8WQC,lisa jon jung,jena,1393545600
11996,11996,6272,B002JCSFSQ,"[2, 2]",5,I have read all seven books in this series. Ap...,"05 16, 2011",AQRSPXLNEQAMA,TerryLP,Peacekeepers Series,1305504000
11997,11997,12483,B0035N1V7K,"[0, 1]",3,This book really just wasn't my cuppa. The si...,"07 26, 2013",A2T5QLT5VXOJAK,hwilson,a little creepy,1374796800
11998,11998,3640,B001W1XT40,"[1, 2]",1,"tried to use it to charge my kindle, it didn't...","09 17, 2013",A28MHD2DDY6DXB,"Allison A. Slater ""Gryphon50""",didn't work,1379376000


In [7]:
df=df[['reviewText','rating']]

In [8]:
df

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4
...,...,...
11995,Valentine cupid is a vampire- Jena and Ian ano...,4
11996,I have read all seven books in this series. Ap...,5
11997,This book really just wasn't my cuppa. The si...,3
11998,"tried to use it to charge my kindle, it didn't...",1


In [9]:
df.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4


In [10]:
df.isnull().sum()

reviewText    0
rating        0
dtype: int64

In [11]:
df['rating'].value_counts()

rating
5    3000
4    3000
3    2000
2    2000
1    2000
Name: count, dtype: int64

In [12]:
df['rating']=df['rating'].apply(lambda x:0 if x<3 else 1)

C:\Users\Swastik Verma\AppData\Local\Temp\ipykernel_24684\4241808129.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rating']=df['rating'].apply(lambda x:0 if x<3 else 1)


In [13]:
df['rating'].value_counts()

rating
1    8000
0    4000
Name: count, dtype: int64

In [14]:
df.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",1
1,Great short read. I didn't want to put it dow...,1
2,I'll start by saying this is the first of four...,1
3,Aggie is Angela Lansbury who carries pocketboo...,1
4,I did not expect this type of book to be in li...,1


# Preprocessing

## Lower Case

In [15]:
df['reviewText']=df['reviewText'].str.lower()

C:\Users\Swastik Verma\AppData\Local\Temp\ipykernel_24684\2784960931.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reviewText']=df['reviewText'].str.lower()


## Remove Special Characters

In [16]:
df['reviewText']

0        jace rankin may be short, but he's nothing to ...
1        great short read.  i didn't want to put it dow...
2        i'll start by saying this is the first of four...
3        aggie is angela lansbury who carries pocketboo...
4        i did not expect this type of book to be in li...
                               ...                        
11995    valentine cupid is a vampire- jena and ian ano...
11996    i have read all seven books in this series. ap...
11997    this book really just wasn't my cuppa.  the si...
11998    tried to use it to charge my kindle, it didn't...
11999    taking instruction is a look into the often hi...
Name: reviewText, Length: 12000, dtype: object

In [17]:
df['reviewText']=df['reviewText'].apply(lambda x:re.sub('[^a-z A-Z 0-9-]', '', x))

C:\Users\Swastik Verma\AppData\Local\Temp\ipykernel_24684\2180374628.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reviewText']=df['reviewText'].apply(lambda x:re.sub('[^a-z A-Z 0-9-]', '', x))


In [18]:
df['reviewText']=df['reviewText'].apply(lambda x: " ".join(x.split()))

C:\Users\Swastik Verma\AppData\Local\Temp\ipykernel_24684\731363273.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reviewText']=df['reviewText'].apply(lambda x: " ".join(x.split()))


## Stop Words

In [19]:
stopwords_set=set(stopwords.words('english'))

In [20]:
df['reviewText']=df['reviewText'].apply(lambda x:" ".join([y for y in x.split() if y not in stopwords_set]))

C:\Users\Swastik Verma\AppData\Local\Temp\ipykernel_24684\3185837085.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reviewText']=df['reviewText'].apply(lambda x:" ".join([y for y in x.split() if y not in stopwords_set]))


## Remove URL

In [21]:
df['reviewText']=df['reviewText'].apply(lambda x:re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , str(x)))

C:\Users\Swastik Verma\AppData\Local\Temp\ipykernel_24684\4266350422.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reviewText']=df['reviewText'].apply(lambda x:re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , str(x)))


## Remove HTML Tag

In [22]:
df['reviewText']=df['reviewText'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text())


C:\Users\Swastik Verma\AppData\Local\Temp\ipykernel_24684\4098260339.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reviewText']=df['reviewText'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text())


In [23]:
df.head()

,reviewText,rating
0,jace rankin may short hes nothing mess man hau...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four books wasnt expect...,1
3,aggie angela lansbury carries pocketbooks inst...,1
4,expect type book library pleased find price right,1


## Lemmatizer

In [24]:
lem=WordNetLemmatizer()

In [ ]:
def lemma(text):
    return(" ".join(lem.lemmatize(word) for word in text.split()))

In [26]:
df['reviewText']=df['reviewText'].apply(lambda x:lemma(x))

C:\Users\Swastik Verma\AppData\Local\Temp\ipykernel_24684\3780451792.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reviewText']=df['reviewText'].apply(lambda x:lemma(x))


## Train Test Split

In [27]:
x_train,x_test,y_train,y_test=train_test_split(df['reviewText'],df['rating'],test_size=0.20)

# NLP

## BOW

In [28]:
bow=CountVectorizer()

In [29]:
x_train_bow=bow.fit_transform(x_train).toarray()
x_test_bow=bow.transform(x_test).toarray()

## TF-IDF

In [30]:
tfidf=TfidfVectorizer()

In [ ]:
x_train_tfidf=tfidf.fit_transform(x_train).toarray()
x_test_tfidf=tfidf.transform(x_test).toarray()

## Word2Vec Custom

In [32]:
x_train_tokens=x_train.apply(lambda x:x.split())
x_test_tokens = x_test.apply(lambda x: x.split())

In [33]:
model=Word2Vec(sentences=x_train_tokens, vector_size=300)

In [34]:
def avg_w2v(doc):
    vec=[model.wv[word] for word in doc if word in model.wv.index_to_key]
    return np.mean(vec, axis=0) if vec else np.zeros(model.vector_size)

In [35]:
x_train_w2v=np.array([avg_w2v(token) for token in x_train_tokens])
x_test_w2v=np.array([avg_w2v(token) for token in x_test_tokens])

## Word2Vec Google

In [ ]:
def avg_w2v_g(doc):
    vec=[Wv[word] for word in doc if word in Wv]
    return np.mean(vec, axis=0) if vec else np.zeros(300)

In [37]:
x_train_w2v_g=np.array([avg_w2v_g(token) for token in x_train_tokens])
x_test_w2v_g=np.array([avg_w2v_g(token) for token in x_test_tokens])

## Checking via Logistic Regression

In [ ]:
nlp_set={
    'BOW':(x_train_bow,x_test_bow),
    'TF-IDF':(x_train_tfidf,x_test_tfidf),
    'Word2Vec(Custom)': (x_train_w2v,x_test_w2v),
    'Word2Vec(Google)': (x_train_w2v_g,x_test_w2v_g)
}

In [49]:
for name,(X_train,X_test) in nlp_set.items():
    lr=LogisticRegression()
    lr.fit(X_train,y_train)
    y_pred=lr.predict(X_test)
    print("=="*35)
    print(name)
    print(accuracy_score(y_test,y_pred))
    print(classification_report(y_test,y_pred))
    print("=="*35)

BOW
0.8345833333333333
              precision    recall  f1-score   support

           0       0.76      0.72      0.74       780
           1       0.87      0.89      0.88      1620

    accuracy                           0.83      2400
   macro avg       0.81      0.80      0.81      2400
weighted avg       0.83      0.83      0.83      2400

TF-IDF
0.8379166666666666
              precision    recall  f1-score   support

           0       0.82      0.65      0.72       780
           1       0.85      0.93      0.89      1620

    accuracy                           0.84      2400
   macro avg       0.83      0.79      0.80      2400
weighted avg       0.84      0.84      0.83      2400

Word2Vec(Custom)
0.7570833333333333
              precision    recall  f1-score   support

           0       0.68      0.48      0.56       780
           1       0.78      0.89      0.83      1620

    accuracy                           0.76      2400
   macro avg       0.73      0.69      0.70

# Model Selection

## TF-IDF

Modela used:  
1. Logistic Regression
2. Multinomial NB
3. Linear SVC
4. Random Forest
4. XGBoost
5. KNN
6. SVC

In [47]:
models={
    "LogisticRegression":LogisticRegression(),
    #"MultinomialNB":MultinomialNB(),
    #"SVC":SVC(),
    "LinearSVC":LinearSVC(),
    "RandomForestClassifier":RandomForestClassifier(),
    "XGBClassifier":XGBClassifier(),
    "KNeighborsClassifier":KNeighborsClassifier()
}

In [44]:
for i in range(len(list(models))):
    mod=list(models.values())[i]
    mod.fit(x_train_tfidf,y_train)
    #Training Data Prediction
    y_pred_train=mod.predict(x_train_tfidf)
    print("=="*35)
    print(list(models.keys())[i])
    print("TRAIN DATA")
    print(accuracy_score(y_train,y_pred_train))
    print(classification_report(y_train,y_pred_train))
    #Test data
    y_pred_test=mod.predict(x_test_tfidf)
    print("TEST DATA")
    print(accuracy_score(y_test,y_pred_test))
    print(classification_report(y_test,y_pred_test))
    print("=="*35)

LogisticRegression
TRAIN DATA
0.9025
              precision    recall  f1-score   support

           0       0.92      0.78      0.84      3220
           1       0.90      0.97      0.93      6380

    accuracy                           0.90      9600
   macro avg       0.91      0.87      0.89      9600
weighted avg       0.90      0.90      0.90      9600

TEST DATA
0.8379166666666666
              precision    recall  f1-score   support

           0       0.82      0.65      0.72       780
           1       0.85      0.93      0.89      1620

    accuracy                           0.84      2400
   macro avg       0.83      0.79      0.80      2400
weighted avg       0.84      0.84      0.83      2400

MultinomialNB
TRAIN DATA
0.7286458333333333
              precision    recall  f1-score   support

           0       1.00      0.19      0.32      3220
           1       0.71      1.00      0.83      6380

    accuracy                           0.73      9600
   macro avg      

## Word2Vec

In [48]:
for i in range(len(list(models))):
    mod=list(models.values())[i]
    mod.fit(x_train_w2v_g,y_train)
    #Training Data Prediction
    y_pred_train=mod.predict(x_train_w2v_g)
    print("=="*35)
    print(list(models.keys())[i])
    print("TRAIN DATA")
    print(accuracy_score(y_train,y_pred_train))
    print(classification_report(y_train,y_pred_train))
    #Test data
    y_pred_test=mod.predict(x_test_w2v_g)
    print("TEST DATA")
    print(accuracy_score(y_test,y_pred_test))
    print(classification_report(y_test,y_pred_test))
    print("=="*35)

LogisticRegression
TRAIN DATA
0.8161458333333333
              precision    recall  f1-score   support

           0       0.78      0.62      0.69      3220
           1       0.83      0.91      0.87      6380

    accuracy                           0.82      9600
   macro avg       0.81      0.77      0.78      9600
weighted avg       0.81      0.82      0.81      9600

TEST DATA
0.82125
              precision    recall  f1-score   support

           0       0.79      0.61      0.69       780
           1       0.83      0.92      0.87      1620

    accuracy                           0.82      2400
   macro avg       0.81      0.77      0.78      2400
weighted avg       0.82      0.82      0.81      2400

LinearSVC
TRAIN DATA
0.8279166666666666
              precision    recall  f1-score   support

           0       0.79      0.67      0.72      3220
           1       0.85      0.91      0.88      6380

    accuracy                           0.83      9600
   macro avg       0.

# HyperParameter Tuning

In [ ]:
lr=LogisticRegression(max_iter=1000)

In [66]:
params = {
    'penalty': ['l2'],
    'C': [10, 1.0, 0.1],
    'solver': ['lbfgs', 'liblinear', 'saga']
}


In [67]:
params

{'penalty': ['l2'],
 'C': [10, 1.0, 0.1],
 'solver': ['lbfgs', 'liblinear', 'saga']}

In [68]:
grid=GridSearchCV(estimator=lr, param_grid=params, scoring='accuracy', cv=5)

In [69]:
grid.fit(x_train_tfidf,y_train)

,estimator,LogisticRegression()
,param_grid,"{'C': [10, 1.0, ...], 'penalty': ['l2'], 'solver': ['lbfgs', 'liblinear', ...]}"
,scoring,'accuracy'
,n_jobs,None
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,penalty,'l2'


In [70]:
    #Training Data Prediction
y_pred_train=grid.predict(x_train_tfidf)
print("=="*35)
print("Logistic Regression")
print("TRAIN DATA")
print(accuracy_score(y_train,y_pred_train))
print(classification_report(y_train,y_pred_train))
    #Test data
y_pred_test=grid.predict(x_test_tfidf)
print("TEST DATA")
print(accuracy_score(y_test,y_pred_test))
print(classification_report(y_test,y_pred_test))
print("=="*35)

Logistic Regression
TRAIN DATA
0.9794791666666667
              precision    recall  f1-score   support

           0       0.97      0.96      0.97      3220
           1       0.98      0.99      0.98      6380

    accuracy                           0.98      9600
   macro avg       0.98      0.98      0.98      9600
weighted avg       0.98      0.98      0.98      9600

TEST DATA
0.8379166666666666
              precision    recall  f1-score   support

           0       0.77      0.71      0.74       780
           1       0.86      0.90      0.88      1620

    accuracy                           0.84      2400
   macro avg       0.82      0.80      0.81      2400
weighted avg       0.84      0.84      0.84      2400



In [71]:
lr1=LogisticRegression(penalty='l2', C=1.0, solver='lbfgs', max_iter=1000)


In [72]:
lr1.fit(x_train_tfidf,y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'


In [74]:
    #Training Data Prediction
y_pred_train=lr1.predict(x_train_tfidf)
print("=="*35)
print("Logistic Regression")
print("TRAIN DATA")
print(accuracy_score(y_train,y_pred_train))
print(classification_report(y_train,y_pred_train))
    #Test data
y_pred_test=lr1.predict(x_test_tfidf)
print("TEST DATA")
print(accuracy_score(y_test,y_pred_test))
print(classification_report(y_test,y_pred_test))
print("=="*35)

Logistic Regression
TRAIN DATA
0.9025
              precision    recall  f1-score   support

           0       0.92      0.78      0.84      3220
           1       0.90      0.97      0.93      6380

    accuracy                           0.90      9600
   macro avg       0.91      0.87      0.89      9600
weighted avg       0.90      0.90      0.90      9600

TEST DATA
0.8379166666666666
              precision    recall  f1-score   support

           0       0.82      0.65      0.72       780
           1       0.85      0.93      0.89      1620

    accuracy                           0.84      2400
   macro avg       0.83      0.79      0.80      2400
weighted avg       0.84      0.84      0.83      2400



In [75]:
lr2=LogisticRegression(penalty='l2', C=0.1, solver='lbfgs', max_iter=1000)


In [76]:
lr2.fit(x_train_tfidf,y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,0.1
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'


In [77]:
    #Training Data Prediction
y_pred_train=lr2.predict(x_train_tfidf)
print("=="*35)
print("Logistic Regression")
print("TRAIN DATA")
print(accuracy_score(y_train,y_pred_train))
print(classification_report(y_train,y_pred_train))
    #Test data
y_pred_test=lr2.predict(x_test_tfidf)
print("TEST DATA")
print(accuracy_score(y_test,y_pred_test))
print(classification_report(y_test,y_pred_test))
print("=="*35)

Logistic Regression
TRAIN DATA
0.74625
              precision    recall  f1-score   support

           0       0.94      0.26      0.41      3220
           1       0.73      0.99      0.84      6380

    accuracy                           0.75      9600
   macro avg       0.83      0.63      0.62      9600
weighted avg       0.80      0.75      0.69      9600

TEST DATA
0.7441666666666666
              precision    recall  f1-score   support

           0       0.89      0.24      0.38       780
           1       0.73      0.99      0.84      1620

    accuracy                           0.74      2400
   macro avg       0.81      0.61      0.61      2400
weighted avg       0.78      0.74      0.69      2400

